In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

2024-03-13 00:56:21.453143: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-13 00:56:21.959300: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 00:56:21.959497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 00:56:22.049014: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-13 00:56:22.230702: I tensorflow/core/platform/cpu_feature_guar

In [2]:
train_df=pd.read_csv('train.csv')

In [3]:
train_df.shape

(8201, 2)

In [4]:
train_df

,FileName,Class
0,1.jpg,0
1,2.jpg,0
2,3.jpg,0
3,4.jpg,0
4,5.jpg,0
...,...,...
8196,8197.jpg,0
8197,8198.jpg,0
8198,8199.jpg,0
8199,8200.jpg,0


In [5]:
train_df.Class.value_counts()

Class
0    4829
1    1405
4     789
3     443
6     295
2     285
7      87
5      68
Name: count, dtype: int64

In [6]:
import os
import cv2

In [7]:
df2 = train_df.sample(n=3000, random_state=42)

In [8]:


# Path to the folder containing images
image_folder = 'train/'

# Create empty lists to store image arrays, filenames, and classes
image_arrays = []
filenames = []
classes = []


In [9]:
df2

,FileName,Class
3567,3568.jpg,0
1322,1323.jpg,0
7757,7758.jpg,0
2407,2408.jpg,0
6328,6329.jpg,1
...,...,...
2676,2677.jpg,1
2142,2143.jpg,4
6825,6826.jpg,7
8113,8114.jpg,0


In [10]:
# Loop through the DataFrame and read/convert each image
for index, row in df2.iterrows():
    filename = row['FileName']
    class_label = row['Class']
    image_path = os.path.join(image_folder, filename)

    # Read the image using OpenCV
    img = cv2.imread(image_path)

    # Convert BGR to RGB (OpenCV loads images in BGR by default)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img=np.array(img)

    # Append the image array, filename, and class to the respective lists
    image_arrays.append(img)
    filenames.append(filename)
    classes.append(class_label)



In [11]:
image_arrays = np.array(image_arrays)

In [12]:
classes=np.array(classes)

In [13]:
image_arrays.shape

(500, 227, 227, 3)

In [14]:
image_arrays[0]

array([[[173, 173, 173],
        [176, 176, 176],
        [178, 178, 178],
        ...,
        [164, 164, 164],
        [165, 165, 165],
        [167, 167, 167]],

       [[172, 172, 172],
        [175, 175, 175],
        [178, 178, 178],
        ...,
        [164, 164, 164],
        [166, 166, 166],
        [168, 168, 168]],

       [[173, 173, 173],
        [175, 175, 175],
        [179, 179, 179],
        ...,
        [161, 161, 161],
        [164, 164, 164],
        [167, 167, 167]],

       ...,

       [[183, 183, 183],
        [171, 171, 171],
        [130, 130, 130],
        ...,
        [183, 183, 183],
        [195, 195, 195],
        [207, 207, 207]],

       [[160, 160, 160],
        [119, 119, 119],
        [ 89,  89,  89],
        ...,
        [187, 187, 187],
        [200, 200, 200],
        [215, 215, 215]],

       [[125, 125, 125],
        [ 90,  90,  90],
        [ 94,  94,  94],
        ...,
        [188, 188, 188],
        [203, 203, 203],
        [220, 220, 220]]

In [15]:
min_value = np.min(image_arrays)
max_value = np.max(image_arrays)

print("Min value:", min_value)
print("Max value:", max_value)

Min value: 0
Max value: 255


In [16]:
from sklearn.preprocessing import MinMaxScaler

# Now, normalized_images will contain your normalized images with values between 0 and 1

# It should have a shape of (8201, 227, 227, 3)

# Reshape the array to (num_images, num_pixels)
num_images, height, width, channels = image_arrays.shape



In [17]:
image_arrays_reshaped = image_arrays.reshape(num_images, -1)

In [18]:
image_arrays_reshaped.shape

(500, 154587)

In [19]:
# Create a MinMaxScaler
scaler = MinMaxScaler()
# Fit the scaler on the flattened data
scaler.fit(image_arrays_reshaped)


MinMaxScaler()

In [20]:
# Transform the data to the range [0, 1]
normalized_images = scaler.transform(image_arrays_reshaped)


In [21]:

# Reshape the normalized data back to the original shape
normalized_images = normalized_images.reshape(num_images, height, width, channels)

In [22]:
normalized_images.shape

(500, 227, 227, 3)

In [23]:
np.min(normalized_images)

0.0

In [24]:
np.max(normalized_images)

1.0000000000000002

In [25]:
X=normalized_images
Y=classes

In [26]:
X.shape

(500, 227, 227, 3)

In [27]:
type(X)

numpy.ndarray

In [28]:
type(Y)

numpy.ndarray

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)

In [31]:
from tensorflow.keras import datasets, layers, models

In [32]:
cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(227, 227, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(input_shape=(227,227,3)),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

2024-03-13 00:57:50.744658: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-13 00:57:50.893463: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-13 00:57:50.894227: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [33]:
cnn.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [34]:
type(Y)

numpy.ndarray

In [36]:
X_test.shape

(100, 227, 227, 3)

In [37]:
X_test[0]

array([[[0.19480519, 0.19480519, 0.19480519],
        [0.1779661 , 0.1779661 , 0.1779661 ],
        [0.1659751 , 0.1659751 , 0.1659751 ],
        ...,
        [0.20087336, 0.20087336, 0.20087336],
        [0.19650655, 0.19650655, 0.19650655],
        [0.19565217, 0.19565217, 0.19565217]],

       [[0.18534483, 0.18534483, 0.18534483],
        [0.1787234 , 0.1787234 , 0.1787234 ],
        [0.16949153, 0.16949153, 0.16949153],
        ...,
        [0.20524017, 0.20524017, 0.20524017],
        [0.20524017, 0.20524017, 0.20524017],
        [0.20524017, 0.20524017, 0.20524017]],

       [[0.17748918, 0.17748918, 0.17748918],
        [0.17226891, 0.17226891, 0.17226891],
        [0.16528926, 0.16528926, 0.16528926],
        ...,
        [0.20434783, 0.20434783, 0.20434783],
        [0.2034632 , 0.2034632 , 0.2034632 ],
        [0.19827586, 0.19827586, 0.19827586]],

       ...,

       [[0.18907563, 0.18907563, 0.18907563],
        [0.19915254, 0.19915254, 0.19915254],
        [0.20762712, 0

In [35]:
cnn.fit(X_train, Y_train, epochs=10)

Epoch 1/10


2024-03-13 00:58:17.337850: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-03-13 00:58:17.706738: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-13 00:58:17.711405: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-13 00:58:17.711581: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:110] Couldn't get ptxas version : FAILED_PRECONDITION: Couldn't get ptxas/nvlink version string: INTERNAL: Couldn't invoke ptxas --version
2024-03-13 00:58:17.714939: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-13 00:58:17.715211: W external/local_xla/xla/stream_executor/gpu/redzone_allocator.cc:322] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptx

InternalError: Graph execution error:

Detected at node Adam/StatefulPartitionedCall_8 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/dinesh/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/dinesh/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1046, in launch_instance

  File "/home/dinesh/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/home/dinesh/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/dinesh/.local/lib/python3.10/site-packages/ipykernel/kernelbase.py", li